In [38]:
using JuMP, Gurobi, LinearAlgebra, Statistics, Plots

In [48]:
#time step
T=[1,2,3,4];
#parent node
a=Dict()
a[4]=[3]
a[3]=[2]
a[2]=[1]
a[1]=nothing
#demand
d=Dict();d[1]=800;d[2]=700;d[3]=1000;d[4]=900
#decision stage
D=[1,2,3];
#capacity and cost list
B=[100,500,1000,1500];C=[247,721,1145,1500];N=length(C);
#limitations, costs and prices
x_bar=1500;y_bar=2000;s_bar=400;
π_p=140;ρ_p=50;ρ_s=30;ρ_w=30;
γ=0;

In [59]:
m=Model(Gurobi.Optimizer)

@variable(m, u[t in D, k in 1:N] >= 0, Int)
#selection of plants with different capacity
@expression(m, x[t in D], sum(u[t,k]*B[k] for k in 1:N))
@expression(m, y[t in D], sum(u[t,k]*C[k] for k in 1:N))

#storage and waste
@variable(m, s[t in T] >= 0, Int)
@variable(m, 0 <= w[t in T], Int)
@constraint(m, [t in D], s[t] <= s_bar)

#cumulative variables
@variable(m, X[t in T] <= x_bar)

# #installation cost constraint
# @constraint(m, sum(y[t] for t in D)<=y_bar)

# helper constraints
@constraint(m, X[1] == 0)
@constraint(m, [t in T[2:end]], X[t] == X[a[t][1]] + x[a[t][1]])

#storing
@constraint(m, s[1] == 0)
@constraint(m, s[T[end]] == 0)
@constraint(m, w[1] == 0)
@constraint(m, [t in T[2:end]], X[t] + s[a[t][1]] - s[t] - w[t] <= d[t])
@constraint(m, [t in T[2:end]], 0 <= X[t] + s[a[t][1]] - s[t] - w[t]) 

#cost
@variable(m, q[t in T])
@constraint(m, [t in D], q[t] == y[t] + ρ_s * s[t] + ρ_p * X[t] + ρ_w * w[t])
@constraint(m,q[T[end]] == ρ_w * w[T[end]] + ρ_p * X[T[end]])
 
#revenue
@variable(m, r[t in T])
@constraint(m, r[1] == 0)
@constraint(m, [t in T[2:end]], r[t] == (π_p * (X[t] + s[a[t][1]] - s[t] - w[t])))

#discounting
@expression(m, α[t in T], 1 / (1 + γ)^(t - 1))

#profit
@expression(m, v[t in T], (r[t] - q[t]) * α[t])
@variable(m, V[t in T])
@constraint(m, V[1] == v[1])
@constraint(m, [t in T[2:end]], V[t] == V[a[t][1]]+v[t])
@expression(m, NPV, V[T[end]])

@objective(m, Max, NPV)
status=optimize!(m)

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 28 rows, 36 columns and 107 nonzeros
Model fingerprint: 0x26f2a856
Variable types: 16 continuous, 20 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e+03, 2e+03]
  RHS range        [4e+02, 1e+03]
Found heuristic solution: objective -0.0000000
Presolve removed 19 rows and 16 columns
Presolve time: 0.00s
Presolved: 9 rows, 20 columns, 37 nonzeros
Variable types: 0 continuous, 20 integer (6 binary)

Root relaxation: objective 2.301000e+05, 4 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 230100.000    0    2   -0.00000 230100.000      -     -    0s
H    0     0                    77500.000000 23010

In [58]:
println("The Profit is ", value.(NPV))
for t in D
        println("At stage ", t, " we should install ", string(value.(x[t])),  " capacity")
        println("At stage ", t, " we stored ", string(value.(s[t])),  " product")
        println("At stage ", t, " we dumped ", string(value.(w[t])),  " product")
end
println("waste is ", value.(s[n]))

The Profit is 229291.0
At stage 1 we should install 800.0 capacity
At stage 1 we stored 0.0 product
At stage 1 we dumped 0.0 product
At stage 2 we should install 100.0 capacity
At stage 2 we stored 100.0 product
At stage 2 we dumped -0.0 product
At stage 3 we should install 0.0 capacity
At stage 3 we stored -0.0 product
At stage 3 we dumped -0.0 product
waste is 0.0
